<img src="logo.png">

# C06. Procedimientos almacenados y disparadores

Recordemos la estructura básica de una función en Postgresql

```sql
CREATE OR REPLACE FUNCTION Nombre(parámetros) 
RETURNS tipo AS $$
DECLARE
declaraciones de las variables;
BEGIN
código ejecutable
END
$$
LANGUAGE plpgsql
```

Ya antes habíamos visto cómo hacer la función suma:

```sql
CREATE OR REPLACE FUNCTION sumar(x int, y int) 
RETURNS int 
AS
$$
DECLARE z int;
BEGIN
z = x + y;
RETURN z;
END
$$
LANGUAGE plpgsql;
```

Otra manera de mandar a llamar a los parámetros de la función es sin declararles un nombre:

```sql
CREATE OR REPLACE FUNCTION multiplicar(int,int) 
RETURNS text 
AS
$$
DECLARE z int;
BEGIN
z = $1 * $2;
RETURN CONCAT('El producto de ',$1,' con ',$2,' es ',z);
END
$$
LANGUAGE plpgsql;
```

Para entender la diferencia entre una función y un procedimiento almacenado en el contexto de las bases de datos, es importante considerar varias propiedades y usos de cada uno.

## Propiedades a Considerar:

- Propósito: Las funciones y procedimientos almacenados tienen diferentes objetivos en una base de datos.
- Valores de Retorno: La capacidad de devolver valores y cómo lo hacen.
- Uso en Consultas: Cómo se utilizan dentro de las consultas SQL.
- Efectos Secundarios: La capacidad de modificar el estado de la base de datos.
- Llamadas y Parámetros: La manera en que se llaman y manejan parámetros.

## Diferencias Principales:

### Funciones:

- Propósito: Generalmente utilizadas para realizar cálculos y devolver un valor.
- Valores de Retorno: Devuelven un solo valor (escalar, tabla, o un tipo complejo).
- Uso en Consultas: Se pueden usar en la cláusula SELECT, WHERE, y otras partes de una consulta SQL.
- Efectos Secundarios: No deben tener efectos secundarios, es decir, no deben modificar el estado de la base de datos.
- Llamadas y Parámetros: Llamadas con sintaxis similar a las funciones en programación (SELECT mi_funcion(param1)).

### Procedimientos Almacenados:

- Propósito: Encapsular una serie de instrucciones SQL para realizar una tarea completa, que puede incluir manipulación de datos.
- Valores de Retorno: No devuelven un valor directamente, pero pueden devolver múltiples valores a través de parámetros de salida.
- Uso en Consultas: No se pueden usar directamente en una cláusula SELECT. Se ejecutan con el comando EXEC o CALL.
- Efectos Secundarios: Pueden tener efectos secundarios, como modificar el estado de la base de datos, insertar, actualizar o eliminar datos.
- Llamadas y Parámetros: Se llaman con una sintaxis específica y pueden aceptar parámetros de entrada y salida (EXEC mi_procedimiento(param1, param2 OUT)).

## Procedimientos almacenados

Para crear un procedimiento almacenado, ocupamos la siguiente sintaxis:

```sql
CREATE OR REPLACE PROCEDURE nombre_del_procedimiento(parámetros)
language plpgsql
as 
$$
begin
cuerpo;
end
$$;
```

Para mandar a llamar el procedimiento, escribimos ``call nombre_del_procedimiento(argumentos)``

**Ejemplo.**

Consideremos la siguiente tabla:

```sql
CREATE TABLE cuentas1(
    id INT GENERATED BY DEFAULT AS IDENTITY,
    cliente VARCHAR(100) NOT NULL,
    balance numeric NOT NULL,
    PRIMARY KEY(id)
);
```

```sql
INSERT INTO cuentas1 (cliente,balance)
VALUES('Jorge',10000),
	  ('Héctor',10000),
	  ('Ramón',10000),
	  ('Luis',10000),
      ('José',10000),
      ('Sofía',10000),
      ('Leticia',10000),
      ('Gloria',10000);
```

El siguiente código crea un procedimiento almacenado llamado *tranferir* que transfiere una cantidad de dinero de una cuenta a otra. Mas precisamente: tomará la cantidad indicada por el tercer parámetro, lo quitará del cliente cuyo id es el primer parámetro y la pasará al balance del cliente cuyo id es es el segundo parámetro.

```sql
create or replace procedure transferir(int, int, numeric)
language plpgsql    
as 
$$
begin
    update cuentas1 set balance = balance - $3 where id = $1;
    update cuentas1 set balance = balance + $3 where id = $2;
end;
$$;
```

``call transferir(x,y,z)`` donde x,y,z son los parámetros

## Triggers

Un trigger  o disparador es un objeto que se almacena en nuestra base de datos y se asocia con una tabla en concreto. Éste trigger se ejecuta cuando sucede algún evento sobre la tabla que está asociada y no es necesario que un usuario lo ejecute

Para crear un trigger en PostgreSQL primero tenemos que crear una función donde el tipo que nos retorna dicha función sea un tipo trigger. A continuación mostramos la sintaxis que debemos de utilizar para crear este tipo de función que retorna un tipo trigger.

```sql
CREATE FUNCTION nombre_funcion()
RETURNS TRIGGER AS $$
BEGIN
acciones a ejecutar;
END;
$$ 
LANGUAGE plpgsql;
```

Una vez que ya tenemos la función que va a ser ejecutada por el trigger, creamos este disparador donde le vamos a indicar cuándo debe activarse. En esta sintaxis se ejecutará una función antes de cada inserción de datos en una tabla especificada.

``` sql 
CREATE TRIGGER nombre_trigger BEFORE INSERT ON tabla FOR EACH ROW EXECUTE PROCEDURE nombre_funcion;
```

**Ejemplo**

Supongamos que queremos mantener un respaldo de cómo era una tabla antes de actualizarla. Creamos primero una tabla con los campos que tiene *cuentas* y otros más:

```sql
create table cuentas_antes(
	id INT GENERATED BY DEFAULT AS IDENTITY,
    cliente VARCHAR(100) NOT NULL,
    balance numeric NOT NULL,
	usuario varchar(250) NOT NULL,
	Fecha date,
	tiempo time,
    PRIMARY KEY(id))
```

El siguiente disparador, llamado *antes_actualizar* se ejecutará antes de que se ejecute cualquier tipo de actualización en la tabla *clientes*, de tal manera que guardará en *cuentas_antes* los registros afectados directamente antes de hacer dicha actualización.    

```sql
create or replace function revisar() returns trigger
as
$$
declare
    Usuario Varchar(250) := user; 
    Fecha date := current_date;
    Tiempo time := current_time;
begin
    INSERT INTO cuentas_antes values (old.id,old.cliente,old.balance, Usuario, Fecha, Tiempo);
    return new;
end
$$
language plpgsql
```    

```sql
create trigger antes_actualizar  before update on cuentas
for each row execute procedure revisar();
```

**Ejemplo**

Supongamos ahora que queremos saber qué nuevos datos se han introducido. Creamos nuevamente una tabla con los campos que tiene *cuentas* y otros más:

```sql 
create table cuentas_ahora(
	id INT GENERATED BY DEFAULT AS IDENTITY,
    cliente VARCHAR(100) NOT NULL,
    balance numeric NOT NULL,
	usuario varchar(250) NOT NULL,
	Fecha date,
	tiempo time,
    PRIMARY KEY(id))
```
	
El siguiente disparador, llamado *Actualizacion_reciente* se ejecutará después de que se ejecute cualquier insertado de datos en la tabla *clientes*, de tal manera que guardará en *cuentas_ahora* los nuevos registros. 

```sql 
create or replace function insertar() returns trigger
as
$$
declare
    Usuario Varchar(250) := user; 
    Fecha date := current_date;
    Tiempo time := current_time;
begin
    INSERT INTO cuentas_ahora values (new.id,new.cliente,new.balance, Usuario, Fecha, Tiempo);
    return new;
end
$$
language plpgsql
```    

```sql
create trigger Actualizacion_reciente  after insert on cuentas
for each row execute procedure insertar();
```
